# Proyecto de Computo Distribuido

---
# Settings

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1714170791855_0001,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
# init spark session
spark = SparkSession.builder.appName('QPP').getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# bucket
NAME = 'javier'
BUCKET = f"s3://itam-analytics-{NAME}"
FOLDER = 'quien_es_quien'
NAME_FILE = 'qqp'

# type of catalog
CATALOG_TYPE = 'medicamentos'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
# Data

In [4]:
# read raw data from S3
df_qqp = spark.read.csv(f"{BUCKET}/{FOLDER}/{NAME_FILE}.csv", header=True, inferSchema=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# look columns
df_qqp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product: string (nullable = true)
 |-- presentation: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- catalog: string (nullable = true)
 |-- price: double (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- store: string (nullable = true)
 |-- type_of_store: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)

In [6]:
# save to parquet in s3, partioned by catalog type
df_qqp.write.mode('overwrite').partitionBy('catalog').parquet(f"{BUCKET}/{FOLDER}/Catalogo")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

---
# Questions

## S1: General

### Get Data

In [7]:
# read parquet from s3
df_qqp = spark.read.parquet(f"{BUCKET}/{FOLDER}/Catalogo")

# look columns
df_qqp.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- product: string (nullable = true)
 |-- presentation: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: double (nullable = true)
 |-- created_at: timestamp (nullable = true)
 |-- store: string (nullable = true)
 |-- type_of_store: string (nullable = true)
 |-- branch: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)
 |-- catalog: string (nullable = true)

In [14]:
# from the column created_at, extract the year
df_qqp = df_qqp.withColumn('year', F.year('created_at'))

# if catalog is in 's/m', 'sin marca' or 'cualquier marca' then set 's/m'
df_qqp = (
    df_qqp
    .withColumn(
        'catalog', F.when(
                col('catalog').isin('s/m', 'sin marca', 'cualquier marca'), 's/m'
            ).otherwise(
                col('catalog')
            )
        )
    )

df_qqp = df_qqp.dropna()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# look # of rows  per year
df_qqp.groupBy('year').count().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------+
|year|   count|
+----+--------+
|2018|16782901|
|2019|13670574|
|2020|10777186|
|2021|12813994|
|2022|14442801|
|2023|14779607|
|2024| 3621415|
+----+--------+

### Q1: ¿Cuántos catálogos diferentes tenemos?

In [10]:
# num different catalogs
df_qqp.agg(F.countDistinct('catalog')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------+
|count(DISTINCT catalog)|
+-----------------------+
|                     12|
+-----------------------+

In [16]:
# num different catalogs per year
df_qqp.groupBy('year').agg(F.countDistinct('catalog')).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------------------+
|year|count(DISTINCT catalog)|
+----+-----------------------+
|2018|                     10|
|2019|                     11|
|2020|                     10|
|2021|                     10|
|2022|                     11|
|2023|                     11|
|2024|                     10|
+----+-----------------------+

En total hay 12 catálogos diferentes, sin embargo no todos los catálogos están presentes en todos los años. Por ejemplo, en 2018 solo hay 10 catálogos diferentes.

### Q2: ¿Cuáles son los 20 catálogos con más observaciones?

In [17]:
# count num of catalogs
(
    df_qqp
    .groupBy('catalog')
    .count()
    .orderBy('count')
    .show()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+--------+
|            catalog|   count|
+-------------------+--------+
|        aeropuertos|     581|
|              tenis|   15750|
|          navidenos|  236352|
|pescados y mariscos|  569571|
|              pacic| 1078659|
|           juguetes| 1431026|
|           mercados| 2237959|
|   utiles escolares| 2935730|
| frutas y legumbres| 5040251|
|  electrodomesticos| 7173020|
|       medicamentos|19215972|
|            basicos|46953607|
+-------------------+--------+

Los productos que más se venden son los básicos, seguido de los medicamentos y electrodométsicos

In [18]:
# table count of each catalog, show only first 20
table_count_catalogue = df_qqp.groupBy('year', 'catalog').count().orderBy('year', 'catalog')

# get only first 20 per year
table_count_catalogue = (
    table_count_catalogue
    .withColumn('rank', F.row_number().over(
        Window.partitionBy().orderBy(F.desc('count'))
        ))
    .filter('rank <= 20')
    )
table_count_catalogue.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------------+-------+----+
|year|           catalog|  count|rank|
+----+------------------+-------+----+
|2018|           basicos|8804761|   1|
|2022|           basicos|7941826|   2|
|2023|           basicos|7740503|   3|
|2019|           basicos|7393082|   4|
|2021|           basicos|6919490|   5|
|2020|           basicos|6169196|   6|
|2018|      medicamentos|3657969|   7|
|2023|      medicamentos|3191800|   8|
|2022|      medicamentos|3176569|   9|
|2021|      medicamentos|2937404|  10|
|2019|      medicamentos|2883869|  11|
|2020|      medicamentos|2570873|  12|
|2024|           basicos|1984749|  13|
|2018| electrodomesticos|1735336|  14|
|2019| electrodomesticos|1350759|  15|
|2023| electrodomesticos|1183127|  16|
|2022| electrodomesticos|1028880|  17|
|2021| electrodomesticos|1004833|  18|
|2018|frutas y legumbres| 938616|  19|
|2022|frutas y legumbres| 925519|  20|
+----+------------------+-------+----+

In [19]:
# save top20 to parquet in s3
table_count_catalogue.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/Top20")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

En una tabla aparte se pueden observar los 20 catalogos con más productos en cada año

### Q3: ¿Tenemos datos de todos los estados del país? De no ser así, ¿cuáles faltan?

In [20]:
# get table count for each year catalog the # of distinct states
table_count_num_states_per_year_catalog = (
    df_qqp
    .groupBy('catalog', 'state', 'year')
    .count()
    .orderBy('catalog', 'state', 'year')
    )

# generate table of # of distinct states per year and catalog
distinct_states = df_qqp.select('state').distinct()
distinct_years = df_qqp.select('year').distinct()
distinct_catalogs = df_qqp.select('catalog').distinct()

df_cross_join_state_year_catalog = (
    distinct_catalogs
    .crossJoin(distinct_states)
    .crossJoin(distinct_years)
    )

# get which catalog state and year are missing
table_missing = (
    df_cross_join_state_year_catalog
    .join(table_count_num_states_per_year_catalog, ['catalog', 'state', 'year'], 'left')
    .filter('count is null')
    )

# show missing
print(f"Missing: {table_missing.count()}")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Missing: 586

Hay 586 combinaciones entre catálogo, estado y año que no tienen información. En una tabla aparte se pueden visualizar cuales son.

In [21]:
# look
table_missing.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------+-------------------+----+-----+
|           catalog|              state|year|count|
+------------------+-------------------+----+-----+
|             pacic|           guerrero|2021| NULL|
|          juguetes|            nayarit|2024| NULL|
|             pacic|       quintana roo|2021| NULL|
|  utiles escolares|            sinaloa|2023| NULL|
|             pacic|          queretaro|2021| NULL|
|             tenis|           tlaxcala|2024| NULL|
|           basicos|            nayarit|2024| NULL|
|             pacic|            durango|2019| NULL|
|  utiles escolares|           guerrero|2023| NULL|
|       aeropuertos|          queretaro|2023| NULL|
|frutas y legumbres|            nayarit|2020| NULL|
|  utiles escolares|             sonora|2020| NULL|
|       aeropuertos|baja california sur|2019| NULL|
|             pacic|          chihuahua|2021| NULL|
|             tenis|    baja california|2024| NULL|
|             tenis|            yucatan|2024| NULL|
|       aero

No hay datos para todos los estados. Faltan 2 estados a partir del 2021.

In [22]:
# get table of the names of the states
table_states = df_qqp.select('state').distinct().orderBy('state')

# count the # of rows per state and year
table_count_states = df_qqp.groupBy('state', 'year').count().orderBy('state', 'year')
table_count_states.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+------+
|              state|year| count|
+-------------------+----+------+
|     aguascalientes|2018|235847|
|     aguascalientes|2019|281560|
|     aguascalientes|2020|216598|
|     aguascalientes|2021|240772|
|     aguascalientes|2022|291820|
|     aguascalientes|2023|313809|
|     aguascalientes|2024| 79726|
|    baja california|2018|619093|
|    baja california|2019|165344|
|    baja california|2020|195002|
|    baja california|2021|104216|
|    baja california|2022|160203|
|    baja california|2023|209715|
|    baja california|2024| 60414|
|baja california sur|2018|163771|
|baja california sur|2019|135558|
|baja california sur|2020|243375|
|baja california sur|2021|283821|
|baja california sur|2022|282548|
|baja california sur|2023|291569|
+-------------------+----+------+
only showing top 20 rows

In [23]:
# pivot the table to have years as columns
(
    table_count_states
    .groupBy('state')
    .pivot('year')
    .agg(F.coalesce(F.lit(1), F.lit(0)))
    .orderBy('state')
    .show(32)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----+----+----+----+----+----+
|               state|2018|2019|2020|2021|2022|2023|2024|
+--------------------+----+----+----+----+----+----+----+
|      aguascalientes|   1|   1|   1|   1|   1|   1|   1|
|     baja california|   1|   1|   1|   1|   1|   1|   1|
| baja california sur|   1|   1|   1|   1|   1|   1|   1|
|            campeche|   1|   1|   1|   1|   1|   1|   1|
|             chiapas|   1|   1|   1|   1|   1|   1|   1|
|           chihuahua|   1|   1|   1|   1|   1|   1|   1|
|    ciudad de mexico|   1|   1|   1|   1|   1|   1|   1|
|coahuila de zaragoza|   1|   1|   1|   1|   1|   1|   1|
|              colima|   1|   1|   1|NULL|NULL|NULL|NULL|
|             durango|   1|   1|   1|   1|   1|   1|   1|
|    estado de mexico|   1|   1|   1|   1|   1|   1|   1|
|          guanajuato|   1|   1|   1|   1|   1|   1|   1|
|            guerrero|   1|   1|   1|   1|   1|   1|   1|
|             hidalgo|   1|   1|   1|   1|   1|   1|   1|
|             

Los estados que no tienen información de ciertos catálogos son los estados de Colima y Nayarit a partir del 2020

### Q4: ¿Cuántas observaciones tenemos por estado?


In [24]:
# count the # of rows per state
df_qqp.groupBy('state').count().orderBy('state').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               state|   count|
+--------------------+--------+
|      aguascalientes| 1660132|
|     baja california| 1513987|
| baja california sur| 1475628|
|            campeche| 1763278|
|             chiapas| 1163732|
|           chihuahua| 1798478|
|    ciudad de mexico|17968798|
|coahuila de zaragoza| 1613314|
|              colima|  560520|
|             durango| 1616227|
|    estado de mexico|12531135|
|          guanajuato| 3330005|
|            guerrero|  981815|
|             hidalgo|  868532|
|             jalisco| 4182990|
| michoacan de ocampo| 1538864|
|             morelos| 1026711|
|             nayarit|  266068|
|          nuevo leon| 3358320|
|              oaxaca| 1536513|
+--------------------+--------+
only showing top 20 rows

La tabla superior muestra los conteos de observaciones por estado. La `ciudad de méxico` es la que predomina

In [25]:
# count the number of different catalogs (WIP)
table_qqp_state_catalog_year = (
    df_qqp
    .groupBy('state', 'catalog', 'year')
    .count()
    .orderBy('state', 'catalog', 'year')
    )
table_qqp_state_catalog_year.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------+----+------+
|         state|           catalog|year| count|
+--------------+------------------+----+------+
|aguascalientes|       aeropuertos|2019|   121|
|aguascalientes|           basicos|2018|106821|
|aguascalientes|           basicos|2019|133114|
|aguascalientes|           basicos|2020|105656|
|aguascalientes|           basicos|2021|112011|
|aguascalientes|           basicos|2022|136357|
|aguascalientes|           basicos|2023|137803|
|aguascalientes|           basicos|2024| 37042|
|aguascalientes| electrodomesticos|2018| 26044|
|aguascalientes| electrodomesticos|2019| 36493|
|aguascalientes| electrodomesticos|2020| 18383|
|aguascalientes| electrodomesticos|2021| 26257|
|aguascalientes| electrodomesticos|2022| 31491|
|aguascalientes| electrodomesticos|2023| 36656|
|aguascalientes| electrodomesticos|2024|  8228|
|aguascalientes|frutas y legumbres|2018| 12824|
|aguascalientes|frutas y legumbres|2019| 14035|
|aguascalientes|frutas y legumbres|2020|

En una tabla por aparte se pueden observar el numero de observaciones por estado, catálogo y año

### Q5: De cada estado obten: el número de catalogos diferentes por año, ¿ha aumentado el número de catálogos con el tiempo?

In [26]:
# count the number of different catalogs per year
table_qqp_year_state_diff_catalog = (
    df_qqp
    .groupBy('state', 'year')
    .agg(F.countDistinct('catalog').alias('count_catalogs'))
    .orderBy('state', 'year')
    .groupBy('state')
    .pivot('year')
    .agg(F.first('count_catalogs'))
    .orderBy('state')
    )
table_qqp_year_state_diff_catalog.show(32)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+----+----+----+----+----+----+
|               state|2018|2019|2020|2021|2022|2023|2024|
+--------------------+----+----+----+----+----+----+----+
|      aguascalientes|   9|  11|   9|  10|  11|  11|   9|
|     baja california|  10|  10|   9|   9|  11|  11|  10|
| baja california sur|   9|  10|   9|  10|  11|  11|  10|
|            campeche|   9|   9|   9|  10|  11|  11|   9|
|             chiapas|  10|  10|   9|   9|  10|  10|   8|
|           chihuahua|  10|  11|   5|  10|  11|  11|  10|
|    ciudad de mexico|  10|  10|   9|  10|  11|  11|  10|
|coahuila de zaragoza|  10|   9|   9|  10|  11|  11|  10|
|              colima|  10|  10|   4|NULL|NULL|NULL|NULL|
|             durango|  10|  10|   9|  10|  11|  11|   9|
|    estado de mexico|  10|  10|   9|  10|  11|  11|  10|
|          guanajuato|  10|  10|   9|  10|  11|  11|   9|
|            guerrero|  10|  10|   8|   8|   8|   7|   9|
|             hidalgo|  10|  11|   9|   9|  10|  11|   8|
|             

Falta observar tendencias en el tiempo para afirmar si hay un crecimiento en el número de catálogos por estado, sin embargo se muestra como para Colima y Nayarit bajó

### Save tables

In [27]:
# save tables to s3
table_count_catalogue.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_count_catalogue")
table_missing.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_missing")
table_qqp_state_catalog_year.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_qqp_state_catalog_year")
table_qqp_year_state_diff_catalog.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/all/table_qqp_year_state_diff_catalog")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## S2: Particular

### Get Data

In [28]:
# read raw data from S3
tbl_medicamentos = spark.read.option("header",True).parquet("s3://itam-analytics-javier/quien_es_quien/Catalogo/catalog=medicamentos/*.parquet")

# look # of rows
tbl_medicamentos.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

19215973

### Q1: ¿Cuańtas marcas diferentes tiene tu categoría?

In [29]:
# how many different brands are there?
tbl_medicamentos.select('brand').distinct().count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40

### Q2: ¿Cuál es la marca con mayor precio? ¿En qué estado?

In [35]:
# which brand has the highest price and in which state?
(
    tbl_medicamentos
    .orderBy(col('price').desc())
    .select('brand', 'state', 'price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+---------+------+
|brand|    state| price|
+-----+---------+------+
|  s/m|chihuahua|5275.0|
+-----+---------+------+
only showing top 1 row

Hay un producto `sin marca` en el estado de `Chihuaha` con un precio de $5,275; el cual sería el máximo

In [32]:
# which brand has the highest price and in which state?
(
    tbl_medicamentos
    .filter(~col('brand').isin('s/m', 'sin marca', 'cualquier marca'))
    .orderBy(col('price').desc())
    .select('brand', 'state', 'price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+-------+
|brand|              state|  price|
+-----+-------------------+-------+
|omron|baja california sur|1999.99|
+-----+-------------------+-------+
only showing top 1 row

El producto `omron` es el producto _identificable_ más caro de QQP, procedente del estadio de `Baja California Sur`

### Q3: ¿Cuál es la marca con menor precio en CDMX? (en aquel entonces Distrito Federal)


In [38]:
# idem but in CDMX
# which brand has the lowest price and in which state?
(
    tbl_medicamentos
    .filter(~col('brand').isin('s/m', 'sin marca', 'cualquier marca'))
    .filter(col('state') == 'ciudad de mexico')
    .orderBy(col('price').asc())
    .select('brand', 'state', 'price').show(1)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----------------+-----+
| brand|           state|price|
+------+----------------+-----+
|protec|ciudad de mexico| 5.21|
+------+----------------+-----+
only showing top 1 row

La marca más barata es Protect, con un precio promedio de $5.21

### Q4: ¿Cuál es la marca con mayores observaciones?

In [40]:
# which brand has more observations?
table_brand_count = tbl_medicamentos.groupBy('brand').count().orderBy('count', ascending=False)
table_brand_count.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               brand|   count|
+--------------------+--------+
|                 s/m|17833565|
|           sin marca|  236467|
|     cualquier marca|  189764|
|              le roy|  164805|
|                zuum|  105381|
|               omron|   80830|
|            medimart|   70578|
|               dalux|   68817|
|              protec|   62108|
|              ensure|   49258|
|         quality day|   39434|
|             nebucor|   31952|
|escudo antibacterial|   30992|
|            vitascom|   30293|
|             curitas|   29325|
|             soriana|   26033|
|inhala care  o ho...|   21635|
|          zuum. klin|   18975|
|           microlife|   18242|
|         3m. nexcare|   15335|
+--------------------+--------+
only showing top 20 rows

La marca _identificable_ con más observaciones es la marca `le roy`

### Q5: ¿Cuáles son el top 5 de marcas con mayor precio en cada estado? ¿Son diferentes?

In [41]:
# for each state, get the top 5 most expensive brands (WIP)
table_price_brand_top5 = (
    tbl_medicamentos
    .groupBy('state', 'brand')
    .agg(F.max('price').alias('max_price'))
    .orderBy(col('max_price').desc())
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy(F.desc('max_price'))
        ))
    .filter('rank <= 5')
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Las top 5 marcas con mayo precio se pueden encontrar en una tabla aparte

### Q6: ¿Cuáles son el top 5 de marcas con menor precio en CDMX? (en aquel entonces Distrito Federal)

In [42]:
# top 5 least expensive brands in CDMX
(
    tbl_medicamentos
    .filter(col('state') == 'ciudad de mexico')
    .groupBy('brand')
    .agg(F.min('price').alias('min_price'))
    .orderBy(col('min_price').asc())
    .select('brand', 'min_price')
    .show(8)
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+
|               brand|min_price|
+--------------------+---------+
|                 s/m|      5.0|
|              protec|     5.21|
|         quality day|      5.4|
|             soriana|      6.0|
|     cualquier marca|      7.5|
|            farmacom|      7.5|
|escudo antibacterial|      9.9|
|            vitascom|     10.0|
+--------------------+---------+
only showing top 8 rows

Las marcas _identificables_ con el menor precio de la CDMX son `protec` `quality day` `soriana` `farmacom` y `escudo antibacterial`

### Q7: ¿Cuáles son el top 5 de marcas con mayores observaciones? ¿Se parecen a las de nivel por estado?

In [43]:
# top 5 brands with more observations per state (WIP)
(
    tbl_medicamentos
    .groupBy('brand')
    .count()
    .orderBy('count', ascending=False)
    .show()
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|               brand|   count|
+--------------------+--------+
|                 s/m|17833565|
|           sin marca|  236467|
|     cualquier marca|  189764|
|              le roy|  164805|
|                zuum|  105381|
|               omron|   80830|
|            medimart|   70578|
|               dalux|   68817|
|              protec|   62108|
|              ensure|   49258|
|         quality day|   39434|
|             nebucor|   31952|
|escudo antibacterial|   30992|
|            vitascom|   30293|
|             curitas|   29325|
|             soriana|   26033|
|inhala care  o ho...|   21635|
|          zuum. klin|   18975|
|           microlife|   18242|
|         3m. nexcare|   15335|
+--------------------+--------+
only showing top 20 rows

Las marcas _identificables_ con el más productos del pais son `le roy` `zuum` `omron` `medimart` y `dalux`

In [44]:
# table of counts by state and brand and get top 5 brands per state
table_brand_state_count_top5 = (
    tbl_medicamentos
    .groupBy('state', 'brand')
    .count()
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy(F.desc('count'))
        ))
    .filter('rank <= 5')
    )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

En una tabla aparte se puede observar las 5 marcas con más observaciones por estado

### Q8: ¿Ha dejado de existir alguna marca durante los años que tienes? ¿Cuál? ¿Cuándo desapareció?

In [47]:
# from the column created_at, extract the year
tbl_medicamentos = tbl_medicamentos.withColumn('year', F.year('created_at'))

# distinct observeations for year and brand (WIP)
table_brand_year_count = (
    tbl_medicamentos
    .groupBy('year', 'brand')
    .count()
    .orderBy('year', 'count')
    .groupBy('brand')
    .pivot('year')
    .agg(F.first('count'))
    .orderBy('brand')
    )

table_brand_year_count.show(40)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----+-------+-------+-------+-------+-------+-------+------+
|               brand|null|   2018|   2019|   2020|   2021|   2022|   2023|  2024|
+--------------------+----+-------+-------+-------+-------+-------+-------+------+
| 3m nexcare. coldhot|NULL|   NULL|   NULL|    291|    463|    366|    390|    90|
|         3m. nexcare|NULL|   NULL|   NULL|   2869|   3697|   3612|   3802|  1355|
|        alfa medical|NULL|   NULL|   NULL|   2265|   1695|   1841|   1828|   443|
|               aurax|NULL|   NULL|   NULL|     37|     35|   NULL|   NULL|  NULL|
|             benesta|NULL|   NULL|   NULL|    378|    771|    811|    543|   110|
|        bumy medical|NULL|   NULL|   NULL|    746|   2722|   2219|   2138|   472|
|          choisemmed|NULL|   NULL|   NULL|    646|   1809|   1950|   2106|   580|
|             citizen|NULL|   NULL|   NULL|   1847|   2872|   2788|   5148|  1587|
|              conair|NULL|   NULL|   NULL|   1079|   2244|   3046|   3801|   953|
|   

In [46]:
(
    tbl_medicamentos
    .groupBy('year', 'brand')
    .count()
    .orderBy('year', 'count')
    .filter('count is not null')
    .groupBy('brand')
    .max('year')
    .filter('max(year) < 2024')
    .show()
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+
|      brand|max(year)|
+-----------+---------+
|      aurax|     2021|
|     jumper|     2021|
|safety mask|     2021|
|   dr. simi|     2021|
|   curapack|     2021|
|     fregon|     2021|
|    duo max|     2021|
+-----------+---------+

En una tabla aparte se puede observar la tabla con las marcas que han dejado de existir y en el año en la que dejó de existir. En su mayoría las que dejaron de existir fue en 2021

### Q9: Genera una gráfica de serie de tiempo por estado para la marca con mayor precio -en todos los años-, donde el eje equis es el año y el eje ye es el precio máximo.

In [48]:
# for each year, get the most expensive brand (WIP)
table_state_year_max_price = (
    tbl_medicamentos
    .groupBy('brand', 'state')
    .agg(F.avg('price').alias('avg_price'))
    .withColumn('rank', F.row_number().over(
        Window.partitionBy('state').orderBy(F.desc('avg_price'))
        ))
    .filter('rank = 1')
    .select('brand','state')
    )
table_state_year_max_price.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+
|               brand|               state|
+--------------------+--------------------+
|                 s/m|                NULL|
|          choisemmed|      aguascalientes|
|home care diagnostic|     baja california|
|home care diagnostic| baja california sur|
|home care diagnostic|            campeche|
|          choisemmed|             chiapas|
|          choisemmed|           chihuahua|
|          choisemmed|    ciudad de mexico|
|          choisemmed|coahuila de zaragoza|
|          choisemmed|              colima|
|              jumper|             durango|
|          choisemmed|    estado de mexico|
|          choisemmed|          guanajuato|
|home care diagnostic|            guerrero|
|home care diagnostic|             hidalgo|
|          choisemmed|             jalisco|
|home care diagnostic| michoacan de ocampo|
|home care diagnostic|             morelos|
|               omron|             nayarit|
|          choisemmed|          

In [49]:
tbl_brand_more_expensive_state = (
    tbl_medicamentos
    .join(table_state_year_max_price, ['brand', 'state'])
    .groupBy('brand', 'state', 'year')
    .agg(F.avg('price').alias('avg_price'))
    )

tbl_brand_more_expensive_state.show(8)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------------+----+------------------+
|               brand|           state|year|         avg_price|
+--------------------+----------------+----+------------------+
|home care diagnostic|         sinaloa|2021| 955.1999999999999|
|              jumper|         durango|2020|            1150.0|
|              jumper|         durango|2021|            1150.0|
|              jumper|        tlaxcala|2020|          1381.625|
|              jumper|    quintana roo|2021|             899.0|
|              jumper|    quintana roo|2020|1061.3333333333333|
|          choisemmed|ciudad de mexico|2022|1003.8672384937239|
|          choisemmed|ciudad de mexico|2024|1002.1666666666666|
+--------------------+----------------+----+------------------+
only showing top 8 rows

En un gráfico por aparte se puede ver la tendencia de los precios máximos por estado en cada año

## Save

In [50]:
# save tables to s3
tbl_brand_more_expensive_state.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_state_year_price")
table_brand_count.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_count")
table_price_brand_top5.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/price_brand_top5")
table_brand_year_count.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_state_count")
table_brand_state_count_top5.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/brand_year_count_top5")
table_state_year_max_price.write.mode('overwrite').parquet(f"{BUCKET}/{FOLDER}/ouputs/state_year_max_price")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…